In [1]:
from IPython.display import Markdown, display # for markdown text
import json # for json methods
import pprint # to print human readable dictionary
import pandas as pd # for visualizations
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer # for TF-IDF

In [2]:
def printmd(string):
    display(Markdown(string))

# FETCHING THE DATA

## All the topics in our database

In [3]:
topics = json.load(open('topics.txt'))
pprint.pprint(topics)

{'16': 'openhardware',
 '18': 'Data Science',
 '19': 'Big Data',
 '20': 'Artificial Intelligence',
 '21': 'Business Intelligence',
 '31': 'arduino',
 '32': 'raspberry pi',
 '33': '3d printer',
 '36': 'Deep Learning',
 '37': 'IoT',
 '38': '3d printing',
 '39': 'open hardware',
 '56': 'Wearable',
 '57': 'Sustainable finance',
 '59': 'Sustainable Finance',
 '60': 'Climate Finance',
 '61': 'Green Bonds',
 '62': 'Green Economy'}


## The parameters in our scenario
We are observing an audience, which is defined by two constraints: a topic and a location. Our example is the audience in Italy interested in the topic: Arduino.

In [4]:
TOPIC_ID=31 # topic = arduino
LOCATION = 'italy'
SIGNAL_STRENGTH = 0 # this value indicates the min number of influencers the retrieved audience members follow within the topic
LIMIT = 50 # number of audience members to consider
TESTING_SET_SIZE=10
HOW_MANY_TWEETS = 50 # amount of most recent tweets (including retweets) to be retrieved to consider in our recommendation engine
INCLUDE_RETWEETS = True

In [5]:
rated_audience_dict = json.load(open('rated_audience.txt'))
unrated_audience_dict = json.load(open('unrated_audience.txt'))

In [6]:
printmd("## An example Twitter profile with all the data fields at this point.")
printmd("### Topic: " + topics[str(TOPIC_ID)])
pprint.pprint(next (iter (rated_audience_dict.values())))

## An example Twitter profile with all the data fields at this point.

### Topic: arduino

{'description': 'Chief Innovation Officer at NTT DATA Italia , proud father of '
                'two beautiful girls, a professor, a technology evangelist, an '
                'holistic thinker and a gentleman.',
 'ground_truth_rating': 1.0,
 'hashtags': 'Milano CheTempoFa Milano CheTempoFa foi13 Cefriel fvw2013 '
             'fvw2013 fvw2013 storytelling Vajont Milano CheTempoFa '
             'StartupWeekend GrandC4Picasso makerfairerome MakerFaireRome '
             'GrandC4Picasso GrandC4Picasso',
 'influencers': '266400754 84094835 767285',
 'location': 'Milan, Italy',
 'screen_name': 'funkysurfer',
 'tweets': 'at Cascina Matiot Disorders of Con Edi Touch is easier than yours '
           'via Startup weekend 3 days at Milan Decoded Milan capital of the '
           'street October 2013 Rain Massima Minima The new science is an open '
           'narration thanks to This project was born via Milan handmade with '
           'Craft Camp of a Pisa ItCup A Roncade in Veneto via Oc

In [7]:
# Computes the TF-IDF values for the given corpus.
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english');
def get_tfidf(corpus):
    return tf.fit_transform(corpus.copy()).todense();

## Separate the data into different arrays

In [8]:
screen_names = [aud['screen_name'] for aud in rated_audience_dict.values()]
influencers_corpus = [aud['influencers'] for aud in rated_audience_dict.values()]
tweets_corpus = [aud['tweets'] for aud in rated_audience_dict.values()]
hashtags_corpus = [aud['hashtags'] for aud in rated_audience_dict.values()]
description_corpus = [aud['description'] for aud in rated_audience_dict.values()]
ground_truth_ratings = np.array([2*aud['ground_truth_rating'] for aud in rated_audience_dict.values()])
#print(tweets_corpus)

In [9]:
# INFLUENCER MATRICES
tfidf_influencer_matrix = np.array(get_tfidf(influencers_corpus)).T;
binary_influencer_matrix = tfidf_influencer_matrix.copy()
binary_influencer_matrix[binary_influencer_matrix>0]=1
# TWEET MATRICES
tfidf_tweets_matrix=np.array(get_tfidf(tweets_corpus)).T;
binary_tweets_matrix = tfidf_tweets_matrix.copy()
binary_tweets_matrix[binary_tweets_matrix>0]=1
# DESCRIPTION MATRICES
tfidf_description_matrix = np.array(get_tfidf(description_corpus)).T;
binary_description_matrix = tfidf_description_matrix.copy()
binary_description_matrix[binary_description_matrix>0]=1
# HASHTAGS MATRICES
tfidf_hashtags_matrix = np.array(get_tfidf(hashtags_corpus)).T;
binary_hashtags_matrix = tfidf_hashtags_matrix.copy()
binary_hashtags_matrix[binary_hashtags_matrix>0]=1
printmd("#### Matrix dimensions")
printmd("Influencer matrix: " + str(tfidf_influencer_matrix.shape))
printmd("Tweets matrix: " +str(tfidf_tweets_matrix.shape))
printmd("Description matrix: " +str(tfidf_description_matrix.shape))
printmd("Hashtags matrix: " +str(tfidf_hashtags_matrix.shape))

#### Matrix dimensions

Influencer matrix: (22, 50)

Tweets matrix: (4859, 50)

Description matrix: (308, 50)

Hashtags matrix: (1158, 50)

# METHODS FOR RECOMMENDATION

# Method A
## Logistic Regression using the similarities coming from user profiling (relaxed parameters)

In [10]:
from tfidf_vectorizer import TwitterAccountSimilarityFinder
import mord as md
from sklearn import datasets, linear_model

In [11]:
from scipy import sparse
def get_user_profile(tfidf_matrix, ratings):
    return np.multiply(tfidf_matrix.T,ratings[:, np.newaxis]).sum(axis=0).reshape(1,tfidf_matrix.shape[0])

In [12]:
def cos_sim(tfidf_matrix, user_profile):
    norm = np.linalg.norm(user_profile);
    return 1.0* tfidf_matrix.T.dot(user_profile.T)/ norm
    

In [13]:
SUBSAMPLING_COUNT = 100 # we will subsample this many times and take the average error for evaluation
rated_audience_dict_ids = list(rated_audience_dict.keys())

In [390]:
mean_square_errors = []
prints_enabled = False
for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): display(ratings)
        
    # FIND USER PROFILES
    influencer_user_profile = get_user_profile(tfidf_influencer_matrix, ratings)
    tweets_user_profile = get_user_profile(tfidf_tweets_matrix, ratings)
    description_user_profile = get_user_profile(tfidf_description_matrix, ratings)
    hashtags_user_profile = get_user_profile(tfidf_hashtags_matrix, ratings)

    # FIND THE COSINE SIMILARITIES
    # THESE WILL THEN BE USED AS FEATURES IN REGRESSION
    influencerSimilarities = cos_sim(tfidf_influencer_matrix, influencer_user_profile)
    tweetSimilarities = cos_sim(tfidf_tweets_matrix, tweets_user_profile)
    descriptionSimilarities = cos_sim(tfidf_description_matrix, description_user_profile)
    hashtagSimilarities = cos_sim(tfidf_hashtags_matrix, hashtags_user_profile)
    
    profile_count = len(rated_audience_dict)
    avgTweetSim = np.mean([e for e in tweetSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his tweets an average similarity score
    avgDescriptionSim = np.mean([e for e in descriptionSimilarities if e!=0]) # if the description of a profile is empty, we assign his descripion an average similarity score
    avgHashtagSim = np.mean([e for e in hashtagSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his hashtags an average similarity score
    
    profiles =[]
    training_indices=[]
    for i in range(profile_count):
        if tweetSimilarities[i]==0: tweetSimilarities[i]= avgTweetSim
        if descriptionSimilarities[i]==0: descriptionSimilarities[i]= avgDescriptionSim
        if hashtagSimilarities[i]==0: hashtagSimilarities[i] = avgHashtagSim
        if ratings[i]!=0: training_indices.append(i)
        profile = {
        'index': i,
        'screen_name':screen_names[i],
        'set': "testing" if ratings[i]==0 else "training",
        'ground_truth':ground_truth_ratings[i],
        'infSim': influencerSimilarities[i].item(0),
        'tweetSim': tweetSimilarities[i].item(0),
        'descSim': descriptionSimilarities[i].item(0),
        'hashtagSim': hashtagSimilarities[i].item(0),
        'score': 0
        }
        count=0
        for key in profile.keys():
            if 'Sim' in key: 
                profile['score']+=profile[key]
                count+=1
        profile['score']/=count*1.0
        profiles.append(profile)

    profiles=pd.DataFrame(profiles)
    if (prints_enabled):
        display(profiles[['screen_name','set','ground_truth','score','infSim','tweetSim','descSim','hashtagSim']].sort_values(by='score',ascending=False).round(2))
        printmd("### Naive approach\n Score is the average of similarities.")
        profiles.plot.scatter('score','ground_truth') # score is the average of the similarities
    
    # Combine the similarities and use them as features to feed to a logistic regressor.
    # uncomment to add the similarity into regression

    featureVectors = influencerSimilarities
    featureVectors = np.column_stack((featureVectors,tweetSimilarities))
    #featureVectors = np.column_stack((featureVectors, descriptionSimilarities))
    #featureVectors = np.column_stack((featureVectors, hashtagSimilarities))

    X = np.array([featureVectors[i] for i in training_indices])
    X_binary = X.copy()
    X_binary[X_binary>0]=1
    
    Y = np.array([ratings[i] for i in training_indices])
    Y_binary = [round(e/10) for e in Y]
    
    # Ordinal Regression
    #classifier = linear_model.LinearRegression()
    # Logistic Regression
    #classifier = md.LogisticIT() #Default parameters: alpha=1.0, verbose=0, maxiter=10000
    
    classifier = linear_model.LogisticRegression(C=1e5)
    #print(X)
    #print(Y)
    classifier.fit(X, Y)

    predictions = classifier.predict(featureVectors)
    profiles['predicted_rating']=predictions
    profiles["squared_error"]=(profiles["ground_truth"]-profiles["predicted_rating"])**2
    if (prints_enabled):
        display(profiles[['screen_name','set','ground_truth', 'predicted_rating','squared_error','score','infSim','tweetSim','descSim','hashtagSim']].sort_values(by='score',ascending=False).round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=profiles.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

Avg Training Mean Squared Error: 2.2795
Avg Testing Mean Squared Error: 14.597


# MATRIX METHODS

In [74]:
import math

In [75]:
# Matrix Factorization via multiplicative update rule
# Original Author: Ali Taylan Cemgil from Bogazici University
def nmf_kl_multiplicative(D, M, W, H, EPOCH=5000):
    MD = D.copy()
    MD[M==0] = 0
    for e in range(EPOCH):
        Xhat = W.dot(H)
        W=W*np.array(((MD/Xhat).dot(H.T)/np.dot(M, H.T)))
        Xhat = W.dot(H)
        H = H*np.array((W.T.dot(MD/Xhat)/np.dot(W.T, M)))
        #print(np.sum(np.abs(MD - M*Xhat))/np.sum(M))
    return W, H

In [76]:
# Regularized Matrix Factorization 
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.15):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        if(i==0):
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        else:         
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        if(k==0):
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
                        else:
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

#         eR = np.dot(P,Q)
#         e = 0
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
#                     for k in range(K):
#                         e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
#         if e < 0.001:
#             break
    return P, Q.T

In [77]:
tfidf_data_matrix =[]
tfidf_data_matrix = tfidf_influencer_matrix # first add influencers
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_tweets_matrix,axis=0) # then tweets
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_description_matrix,axis=0) # then descriptions
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_hashtags_matrix,axis=0) # then hashtags

binary_data_matrix =[]
binary_data_matrix = binary_influencer_matrix # first add influencers
binary_data_matrix = np.append(binary_data_matrix,binary_tweets_matrix,axis=0) # then tweets
binary_data_matrix = np.append(binary_data_matrix,binary_description_matrix,axis=0) # then descriptions
binary_data_matrix = np.append(binary_data_matrix,binary_hashtags_matrix,axis=0) # then hashtags

#display(pd.DataFrame(binary_data_matrix))

# Method B 
## Non-negative Matrix Factorization (NMF) without the ratings row = Document Clustering

In [78]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]
Nc = tfidf_data_matrix.shape[1]

for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    # Initialize W and H with random numbers
    W = np.random.rand(Nr, R)*100
    H = np.random.rand(R, Nc)*100

    Mask = np.ones_like(tfidf_data_matrix)
    Mask[np.isnan(tfidf_data_matrix)] = 0

    W,H = nmf_kl_multiplicative(tfidf_data_matrix, Mask, W, H, EPOCH=1)

    # Cluster numbers holds which cluster each user is assigned to
    cluster_numbers = np.argmax(H,axis=0)
    # Ratings each cluster is assigned to (initialized to zeros)
    cluster_ratings = np.zeros(R)

    for i in range(len(cluster_numbers)):
        cluster_no = cluster_numbers[i]
        cluster_ratings[cluster_no]+=1.0*ratings[i] 

    for i in range(R):
        cluster_size = float(len(cluster_numbers[cluster_numbers==i]))
        if cluster_size!=0:
            cluster_ratings[i]/=cluster_size
   
    predictions = [np.around(cluster_ratings[cluster_no]) for cluster_no in cluster_numbers]    
    # Found the cluster ratings
    cluster_ratings = [{'cluster_no':i, 'rating':cluster_ratings[i]} for i in range(R)]
    cluster_ratings = pd.DataFrame(cluster_ratings).round(2)
    if (prints_enabled): 
        printmd("Cluster numbers:")
        display(cluster_numbers)
        printmd("Cluster ratings:")
        display(cluster_ratings)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))
    

Avg Training Mean Squared Error: 6.7595
Avg Testing Mean Squared Error: 10.066


# Method D 
## Non-negative Matrix Factorization (NMF) with the ratings row

In [79]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]+1
Nc = tfidf_data_matrix.shape[1]

for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_data_matrix,axis=0)

    # Initialize W and H with random numbers
    W = np.random.rand(Nr, R)*100
    H = np.random.rand(R, Nc)*100

    Mask = np.ones_like(tfidf_data_matrix_with_ratings)
    Mask[np.isnan(tfidf_data_matrix_with_ratings)] = 0

    W,H = nmf_kl_multiplicative(tfidf_data_matrix_with_ratings, Mask, W, H, EPOCH=1)
    Xhat = W.dot(H)
    predictions = Xhat[0]
    predictions = np.around(10.0*predictions/max(predictions))
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

Avg Training Mean Squared Error: 7.97125
Avg Testing Mean Squared Error: 8.415


In [80]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]+1
Nc = tfidf_data_matrix.shape[1]
for T in np.arange(1,10,0.1):
    for iteration in range(1):
        predictions=[]
        TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
        #if (prints_enabled): print(TESTING_SET_IDS)
        ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
        if (prints_enabled): 
            printmd("Ratings:")
            display(ratings)

        tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
        tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_data_matrix,axis=0)

        # Initialize W and H with random numbers
        W = np.random.rand(Nr, R)*100
        H = np.random.rand(R, Nc)*100

        Mask = np.ones_like(tfidf_data_matrix_with_ratings)
        Mask[np.isnan(tfidf_data_matrix_with_ratings)] = 0

        W,H = nmf_kl_multiplicative(tfidf_data_matrix_with_ratings, Mask, W, H, EPOCH=1)
        Xhat = W.dot(H)
        predictions = Xhat[0]
        predictions = np.around(10.0*predictions/max(predictions))


        results = pd.DataFrame()
        results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
        results['ground_truth']=[0 if r<=T else 1 for r in ground_truth_ratings] # binary ground truth
        results['predicted_rating']=[0 if r<=T else 1 for r in predictions] # binary predictions
        results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2

        if (prints_enabled):
            display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

        evaluation = pd.DataFrame()
        evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
        mean_square_error = {
            'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
            'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
            }
        mean_square_errors.append(mean_square_error)
        
    #Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
    Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
    #print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
    print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing.round(2)) + " Threshold: " + str(T))
    


Avg Testing Mean Squared Error: 0.0 Threshold: 1.0
Avg Testing Mean Squared Error: 0.1 Threshold: 1.1
Avg Testing Mean Squared Error: 0.13 Threshold: 1.2
Avg Testing Mean Squared Error: 0.12 Threshold: 1.3
Avg Testing Mean Squared Error: 0.1 Threshold: 1.4
Avg Testing Mean Squared Error: 0.12 Threshold: 1.5
Avg Testing Mean Squared Error: 0.1 Threshold: 1.6
Avg Testing Mean Squared Error: 0.09 Threshold: 1.7
Avg Testing Mean Squared Error: 0.09 Threshold: 1.8
Avg Testing Mean Squared Error: 0.1 Threshold: 1.9
Avg Testing Mean Squared Error: 0.1 Threshold: 2.0
Avg Testing Mean Squared Error: 0.13 Threshold: 2.1
Avg Testing Mean Squared Error: 0.16 Threshold: 2.2
Avg Testing Mean Squared Error: 0.17 Threshold: 2.3
Avg Testing Mean Squared Error: 0.18 Threshold: 2.4
Avg Testing Mean Squared Error: 0.19 Threshold: 2.5
Avg Testing Mean Squared Error: 0.2 Threshold: 2.6
Avg Testing Mean Squared Error: 0.21 Threshold: 2.7
Avg Testing Mean Squared Error: 0.21 Threshold: 2.8
Avg Testing Mean Sq

In [82]:
mean_square_errors = []
prints_enabled = False
K = 16
for iteration in range(5):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_influencer_matrix,axis=0)

    # Initialize W and H with random numbers    
    R = tfidf_data_matrix_with_ratings
    N= len(R)
    M = len(R[0])
    steps = 2000

    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K,steps)
    nR = np.dot(nP, nQ.T)

    predictions =np.round(nR[0].reshape(1,len(nR[0])))[0]
    print(predictions)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

[  2.   5.   6.   4.   5.  10.  10.   9.   3.   5.   6.   8.   6.   2.   2.
   2.   5.   7.   2.   5.   8.   3.   1.   4.   7.   2.   7.   4.   4.   3.
   2.   7.   5.   5.   2.   7.   1.   4.   2.   6.   4.   2.   4.   3.   2.
   6.   5.  10.   8.   5.]
[  2.   1.   6.   4.   5.  10.  10.   3.   4.   5.   6.   8.   6.   3.   2.
   2.   5.   2.   2.   5.   8.   4.   3.   4.   7.   9.   5.   4.   3.   3.
   2.   7.   5.   6.   2.   7.   1.   4.   2.   8.   4.   2.   4.   3.   4.
   6.   5.  10.   5.   5.]
[  2.   4.   6.   4.   5.  10.  10.   9.   3.   5.   6.   8.   7.   3.   2.
   2.   6.   2.   2.   5.   8.   2.   1.   6.   8.   9.   5.   4.   4.   3.
   2.   5.   5.   6.   2.   7.   1.   4.   3.   8.   4.   2.   2.   2.   2.
   6.   7.  10.   7.   5.]
[  2.   1.   6.   4.   4.  10.   6.   9.   3.   5.   6.   8.   7.   3.   2.
   4.   5.   2.   2.   5.   8.   2.   1.   4.   7.   2.   5.   4.   4.   3.
   2.   4.   5.   6.   2.   7.   1.   4.   2.   8.   4.   2.   4.   2.   2.
   2.  

In [38]:
mean_square_errors = []
prints_enabled = False
K = 16
for iteration in range(5):
    print(str(iteration))
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)
        
    # FIND USER PROFILES
    influencer_user_profile = get_user_profile(tfidf_influencer_matrix, ratings)
    tweets_user_profile = get_user_profile(tfidf_tweets_matrix, ratings)
    description_user_profile = get_user_profile(tfidf_description_matrix, ratings)
    hashtags_user_profile = get_user_profile(tfidf_hashtags_matrix, ratings)

    # FIND THE COSINE SIMILARITIES
    # THESE WILL THEN BE USED AS FEATURES IN REGRESSION
    influencerSimilarities = cos_sim(tfidf_influencer_matrix, influencer_user_profile)
    tweetSimilarities = cos_sim(tfidf_tweets_matrix, tweets_user_profile)
    descriptionSimilarities = cos_sim(tfidf_description_matrix, description_user_profile)
    hashtagSimilarities = cos_sim(tfidf_hashtags_matrix, hashtags_user_profile)
    
    avgTweetSim = np.mean([e for e in tweetSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his tweets an average similarity score
    avgDescriptionSim = np.mean([e for e in descriptionSimilarities if e!=0]) # if the description of a profile is empty, we assign his descripion an average similarity score
    avgHashtagSim = np.mean([e for e in hashtagSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his hashtags an average similarity score
    tweetSimilarities[tweetSimilarities==0]=avgTweetSim
    descriptionSimilarities[descriptionSimilarities==0]=avgDescriptionSim
    hashtagSimilarities[hashtagSimilarities==0]=avgHashtagSim

    
    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    #tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,influencerSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tweetSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,descriptionSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,hashtagSimilarities.T,axis=0)
    display(pd.DataFrame(tfidf_data_matrix_with_ratings))

    # Initialize W and H with random numbers    
    R = tfidf_data_matrix_with_ratings
    N= len(R)
    M = len(R[0])
    steps = 2000

    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K,steps)
    nR = np.dot(nP, nQ.T)

    predictions =np.round(nR[0].reshape(1,len(nR[0])))[0]
    #print(predictions)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    #if (prints_enabled):
    #display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

0


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,6.000000,4.000000,0.000000,10.000000,0.000000,10.000000,3.000000,5.000000,...,0.000000,2.000000,4.000000,3.000000,2.000000,6.000000,5.000000,10.000000,8.000000,0.000000
1,0.192706,0.240506,0.389595,0.175621,0.141782,0.277276,0.145564,0.460521,0.200435,0.236350,...,0.132184,0.334859,0.200776,0.162068,0.298741,0.224321,0.234655,0.394256,0.302406,0.078900
2,0.139163,0.173881,0.200978,0.221400,0.109634,0.368019,0.045505,0.302445,0.238862,0.202050,...,0.071476,0.055124,0.128105,0.182120,0.055124,0.272034,0.237712,0.329553,0.282231,0.080639
3,0.061438,0.036904,0.199247,0.121632,0.011146,0.295976,0.007954,0.316257,0.103118,0.235409,...,0.128500,0.057317,0.187036,0.085975,0.061630,0.171950,0.143292,0.288734,0.242066,0.031969


1


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,1.000000,6.000000,4.000000,0.000000,10.000000,10.000000,10.000000,3.000000,5.000000,...,0.000000,2.000000,4.000000,3.000000,2.000000,6.000000,5.000000,10.000000,8.000000,5.000000
1,0.155961,0.235962,0.384045,0.174419,0.142694,0.263902,0.331155,0.438502,0.197984,0.239174,...,0.132277,0.328151,0.199376,0.154619,0.296571,0.211100,0.220004,0.381867,0.287252,0.181051
2,0.073145,0.169681,0.182247,0.227162,0.138193,0.334859,0.294925,0.287423,0.255600,0.171542,...,0.076152,0.053678,0.137492,0.196108,0.053678,0.275395,0.215758,0.350057,0.264569,0.213710
3,0.003867,0.032388,0.189206,0.114132,0.012052,0.277728,0.277178,0.298355,0.095578,0.235009,...,0.001316,0.053783,0.196448,0.080674,0.065165,0.164343,0.134457,0.270932,0.225759,0.162922


2


...

3


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,6.000000,0.000000,5.000000,10.000000,10.000000,0.000000,3.000000,5.000000,...,4.000000,2.000000,4.000000,0.000000,2.000000,6.000000,5.000000,10.000000,8.000000,5.000000
1,0.209630,0.224691,0.338816,0.104399,0.244693,0.271442,0.350216,0.241319,0.215803,0.256999,...,0.224815,0.254023,0.214810,0.100295,0.235410,0.229383,0.251161,0.374830,0.311529,0.200267
2,0.130422,0.178311,0.181417,0.112768,0.263131,0.354561,0.274549,0.045663,0.272949,0.194833,...,0.175878,0.054028,0.153168,0.098884,0.054028,0.295525,0.235917,0.305049,0.283722,0.237947
3,0.060703,0.036463,0.193510,0.006915,0.151548,0.288336,0.291016,0.023481,0.104238,0.246598,...,0.114649,0.056631,0.177736,0.130390,0.061535,0.173047,0.141578,0.285281,0.240547,0.174988


4


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,1.000000,0.000000,4.000000,5.000000,10.000000,10.000000,10.000000,3.000000,5.000000,...,4.000000,2.000000,4.000000,3.000000,2.000000,6.000000,0.000000,10.000000,0.000000,5.000000
1,0.171442,0.216602,0.263576,0.189426,0.243754,0.276062,0.345727,0.403647,0.213661,0.253618,...,0.214532,0.271190,0.206495,0.164442,0.244127,0.235620,0.124376,0.382271,0.143725,0.201314
2,0.079847,0.179417,0.045859,0.225916,0.233180,0.309173,0.282802,0.273510,0.242418,0.214932,...,0.150084,0.051472,0.153314,0.188800,0.051472,0.248321,0.112527,0.352219,0.078936,0.203012
3,0.003948,0.035349,0.028417,0.116505,0.151124,0.283501,0.282069,0.307685,0.101052,0.239386,...,0.113691,0.054901,0.196643,0.082351,0.066519,0.164703,0.133281,0.276564,0.014008,0.169063


Avg Training Mean Squared Error: 0.0
Avg Testing Mean Squared Error: 6.44
